In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import calendar

from sklearn.preprocessing import PolynomialFeatures
from collections import Counter, defaultdict
from sklearn.metrics import roc_curve, auc
from sklearn import preprocessing
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

In [2]:
dataset = pd.read_csv('network_backup_dataset.csv')
dataset = dataset.drop(['Backup Time (hour)'],axis=1)

In [3]:
def getX_Y_from_dataset(dataset):
    Y = (dataset['Size of Backup (GB)'].as_matrix())
    X = dataset.drop(['Size of Backup (GB)'],axis=1)
    return X,Y

In [16]:
def encode_day_names(days):
    day_to_num = dict(zip(list(calendar.day_name), range(1, 8)))
    return [day_to_num[day] for day in days]
def encode_datalist(datalist):
    encoding = {}
    for i in datalist:
        if i in encoding:
            continue
        else:
            encoding[i] = i.split('_')[-1]
    return [encoding[i] for i in datalist]
def scalar_encode(dataset):
    dataset_copy = dataset.copy()
    # dataset_copy contains scalar encoding of features
    dataset_copy['Day of Week'] = encode_day_names(dataset_copy['Day of Week'])
    dataset_copy['File Name'] = encode_datalist(dataset_copy['File Name'])
    dataset_copy['Work-Flow-ID'] = encode_datalist(dataset_copy['Work-Flow-ID'])
    return dataset_copy

    
def one_hot_encode(dataset):
    dataset_cat = dataset.select_dtypes(include=[object])
    dataset_cat = scalar_encode(dataset_cat)
    oh_enc = preprocessing.OneHotEncoder()
    oh_enc.fit(dataset_cat)
    onehotlabels = oh_enc.transform(dataset_cat)
    cat_array = onehotlabels.toarray()
    dataset_no_cat = dataset.select_dtypes(exclude=[object])
    non_cat_array = dataset_no_cat.as_matrix()
    return np.concatenate((non_cat_array,cat_array),axis = 1)
    

In [17]:
def perform_10fold(X,y,regressor):
    kf = KFold(n_splits=10,shuffle = True,random_state = 42)
    i = 1
    bestModel = None
    tr_e = 0
    ts_e = 0
    min_ts_e = 10
    for train_index, test_index in kf.split(X):
        #print("Fold : ",i)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        regressor.fit(X_train,y_train)
        train_preds = regressor.predict(X_train)
        test_preds = regressor.predict(X_test)
        test_error = mean_squared_error(y_test,test_preds)
        tr_e += mean_squared_error(y_train,train_preds)
        ts_e += test_error
        if(test_error<min_ts_e):
            min_ts_e = test_error
            bestModel = regressor
        i = i+1
    return np.sqrt(tr_e/10),np.sqrt(ts_e/10), bestModel

In [18]:
X,Y= getX_Y_from_dataset(dataset)

In [19]:
X =one_hot_encode(X)

In [21]:
def analyse_nn_regressor(layers,max_layer_size,interval):
    train_map = defaultdict(list)
    test_map = defaultdict(list)
    for activation_ in ['identity', 'logistic', 'tanh', 'relu']:
        print('activation function:',activation_)
        for hidden_size in range(1,max_layer_size,interval):
            mlp_reg = MLPRegressor(hidden_layer_sizes=tuple([hidden_size]*layers),activation=activation_,random_state=1)
            train_rmses,test_rmses,m=perform_10fold(X,Y,mlp_reg)
            print(train_rmses,test_rmses)
            train_map[activation_].append(train_rmses)
            test_map[activation_].append(test_rmses)
    for algo in test_map:
        plt.plot(range(1,max_layer_size,interval),test_map[algo], label = algo, marker='.',markersize=4)
    plt.title('Neural Network Regression Performance | size : layers ='+str(layers))
    plt.xlabel('Size of each layer')
    plt.ylabel('Test RMSE')
    plt.legend()
    plt.show()
            

In [ ]:
analyse_nn_regressor(1,200,2)

activation function: identity
0.0898448566956 0.0900431661475
0.0899262903226 0.0901458635176
0.0902705817951 0.0904654536543
0.0898210283725 0.0899923300898
0.089881983097 0.0900418516429
0.0898582151076 0.0900491691642
0.0898109598505 0.0899770914158
0.0898005481209 0.0899501930242
0.0914173733706 0.0914318256165
0.0898579748263 0.0900766975813
0.090226977646 0.0905892797625
0.089894212014 0.0900831687463
0.0899775829023 0.0901441386472
0.0898213203609 0.0900056322279
0.0898601015215 0.090057193675
0.0910669889099 0.0914484699867
0.0898269074348 0.0900363512197
0.0898380999907 0.0900067341256
0.0899394485891 0.0901395317742
0.0899846039196 0.0902938456629
0.0899802182389 0.0901558619676
0.089886550868 0.0901116020153
0.089962610626 0.0902204237087
0.0899814248502 0.0900930237614
0.08990801531 0.0902364562565
0.0898683779741 0.0900266596789
0.0898616405251 0.0900273491634
0.0899027741122 0.0900942510256
0.0907097472562 0.0909059186062
0.0901339558172 0.0904156220144
0.0900040619268 0.

0.0516370032147 0.0531586137543
0.049743430599 0.05109536252
0.0499887170704 0.0513525931248
0.0507176305718 0.0520158074968
0.0508568445424 0.0520027485862
0.083364902813 0.0841816120803
0.050770575578 0.0524172036338
0.0513833825679 0.052635276511
0.0779654266198 0.0801611453748
0.0515419535873 0.0528729365213
0.0514600027786 0.0527744946127
0.0746716837128 0.0766382385089
0.0840287174528 0.0848134269118
0.0502494943653 0.0515657390736
0.0531603412334 0.0544393390046
0.0522754092055 0.0537868344231
0.0510333782633 0.0523130694054
0.0516866683845 0.0530992478404
0.0513639736752 0.0524641064686
0.0500962996429 0.0516479409999
0.0506256849448 0.0518371798396
0.0526431534717 0.053963964991
0.052008629036 0.0533866666858
0.083227321755 0.0841375136779
0.0511250712341 0.0523676841187
0.0653408348285 0.0657366634803
0.0512857306628 0.0523165465287
0.0515819020009 0.0528165179556
0.0507450961229 0.0517779336301
0.053177827595 0.0542922291869
0.0808581866453 0.0814965563946


In [ ]:
analyse_nn_regressor(2,100,2)

activation function: identity
0.0723762566094 0.0725763577688
0.0718288159377 0.071945034621
0.0762881112378 0.0764434642414
0.0715618348201 0.0717788760484
0.071492170745 0.0717098789875
0.0714892844189 0.071714415104
0.0716843830978 0.0719999198473
0.0715426557765 0.071769593634
0.0714931730582 0.0716908873776
0.0714480829977 0.0716134445329
0.0721235435723 0.0721203639803
0.0717189680393 0.0718752745359
0.07159845885 0.0717108890534
0.0719851138223 0.0723481669648
0.0717980531202 0.0719200688914
0.0717661848208 0.0719403048752
0.0722232628418 0.0723641633245
0.0718449489492 0.0720414434774
0.0718708243205 0.0722222757101
0.0748660689612 0.0748988015291
0.0718452051819 0.0720192939819
0.0726202723131 0.0727661338194
0.0719315571468 0.0720737370501
0.0718228551775 0.0719637303051
0.0716872091928 0.0718891691885
0.0736352381209 0.0737391667073
0.0722238578693 0.0724427234986
0.0718474514015 0.0720175714785
0.0718610655766 0.0720826816264
0.0738965249576 0.0743614903527
0.0714831289084 

In [ ]:
analyse_nn_regressor(3,50,2)

In [ ]:
analyse_nn_regressor(5,20,2)

## KNN

In [ ]:
dataset.columns

In [ ]:
def analyze_knn(neighbor_range,remove_cols=[]):
    removed_data = dataset.drop(remove_cols,axis=1)
    X_r,Y_r = getX_Y_from_dataset(removed_data)
    X_r = one_hot_encode(X_r)
    errors = []

    for neighbors in neighbor_range:
        print("Neighbors:",neighbors)
        knn_model = KNeighborsRegressor(n_neighbors = neighbors )
        tr,ts,m = perform_10fold(X_r,Y_r,knn_model)
        errors.append(ts)
    plt.title('KNN Regression Performance excluding columns '+str(remove_cols))
    plt.xlabel('Neighbors')
    plt.ylabel('Test RMSE Error')
    plt.plot(neighbor_range,errors,marker='.',markersize=4)
    plt.show()
    
    

In [ ]:
analyze_knn(range(2,100,2))

In [ ]:
analyze_knn(range(2,100,2),['Week #','File Name'])

In [ ]:
workflows = list(set(dataset['Work-Flow-ID'].values))
workflow_datasets=[dataset.loc[dataset['Work-Flow-ID'] == x] for x in  workflows]
workflow_datasets_xy = [getX_Y_from_dataset(d) for d in workflow_datasets] 
workflow_datasets_xy_oh = [(one_hot_encode(x[0]),x[1]) for x in workflow_datasets_xy]

### Part 2d

In [ ]:
workflow_datasets_xy_oh = [(scalar_encode(x[0]),x[1]) for x in workflow_datasets_xy]

In [ ]:
def plot_graph(x,y,xlabel,ylabel,title):
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.scatter(x, y, color='blue', lw=1, label=y,s=3)
    #plt.plot([y.min(), y.max()], [0,0], 'k--', lw=4)
    plt.show()

In [ ]:
def plot_graph_time(y1,y2,y1label,y2label,ylabel,title):
    plt.title(title)
    plt.scatter(np.arange(y1.shape[0]), y1, color='blue', label=y1label,s=3)
    plt.scatter(np.arange(y1.shape[0]), y2, color='red', label=y2label,s=3)
    plt.xlabel('Time')
    plt.ylabel(ylabel)
    plt.legend(loc='upper left')
    plt.show()

### 2d (i)

In [ ]:
def analyse_lin_reg_separate(plots=False):
    i = 0
    for x in workflow_datasets_xy_oh:
        print(workflows[i])
        i+=1
        regressor = LinearRegression()
        avg_tr,avg_ts,bestModel = perform_10fold(x[0],x[1],regressor)
        print("------------------------------")
        print("Average Training RMSE : ",avg_tr)
        print("Average Test RMSE : ",avg_ts)
        predicted = bestModel.predict(x[0])
        if(plots):
            plot_graph(x[1],predicted,"Actual","Fitted","Fitted vs Actual")
            plot_graph_time(x[1],predicted,"Actual","Fitted","Actual and Fitted values","Actual and Fitted values over time")
            plot_graph(predicted,x[1]-predicted,"Fitted","Residual","Residual vs Fitted")
            plot_graph_time(x[1]-predicted,predicted,"Residual","Fitted","Residual and Fitted values","Residual and Fitted values over time")


In [ ]:
def analyse_poly_lin_reg_separate(plots=False,poly_degrees = []):
    i = 0
    test_rm = defaultdict(list)
    train_rm= defaultdict(list)
    best_deg_map = {}
    for x in workflow_datasets_xy_oh:
        print("****************************")
        print(workflows[i])        
        best_ts = 10000
        best_model = None
        best_deg = -1
        best_poly = None
        for deg in poly_degrees:
            regressor = LinearRegression()
            poly = PolynomialFeatures(deg)
            x_ = poly.fit_transform(x[0])
            avg_tr,avg_ts,bestModel = perform_10fold(x_,x[1],regressor)
            test_rm[workflows[i]].append(avg_ts)
            train_rm[workflows[i]].append(avg_ts)
            print("average Training RMSE : ",avg_tr)
            print("average Test RMSE : ",avg_ts)
            if(avg_ts<=best_ts):
                best_ts = avg_ts
                best_model = bestModel
                best_deg = deg
                best_poly = poly
                print('best so far',deg)
        x_tot = best_poly.transform(x[0])
        predicted = best_model.predict(x_tot)
        best_deg_map[workflows[i]]=best_deg
        if(plots):
            plot_graph(x[1],predicted,"Actual","Fitted",str(workflows[i])+" Fitted vs Actual")
            plot_graph_time(x[1],predicted,"Actual","Fitted","Actual and Fitted values",str(workflows[i])+" Actual and Fitted values over time")
            plot_graph(predicted,x[1]-predicted,"Fitted","Residual",str(workflows[i])+" Residual vs Fitted")
            plot_graph_time(x[1]-predicted,predicted,"Residual","Fitted","Residual and Fitted values",str(workflows[i])+" Residual and Fitted values over time")
        i+=1
    print(best_deg)
    for x in workflows:
        plt.plot(poly_degrees,test_rm[x], label = x, marker='.',markersize=4)
    plt.title('Avg Test RMSE Vs Polynomial Degree')
    plt.xlabel('Degree of Polynomial')
    plt.ylabel('Test RMSE')
    plt.legend()
    plt.show()
    for x in workflows:
        plt.plot(poly_degrees,train_rm[x], label = x, marker='.',markersize=4)
    plt.title('Avg Train RMSE Vs Polynomial Degree')
    plt.xlabel('Degree of Polynomial')
    plt.ylabel('Train RMSE')
    plt.legend()
    plt.show()
    
        
        

In [ ]:
analyse_poly_lin_reg_separate(poly_degrees=[2,3,4,5,6,7,8])